# Tutorial 3: Running Quantization-Aware Training (QAT) on Bert

In this tutorial, we'll build on top of Tutorial 2 by taking the Bert model fine tuned for sequence classification and running Mase's quantization pass. First, we'll run simple Post-Training Quantization (PTQ) and see how much accuracy drops. Then, we'll run some further training iterations of the quantized model (i.e. QAT) and see whether the accuracy of the trained quantized model approaches the accuracy of the original (full-precision) model.

In [ ]:
from pathlib import Path
from chop import MaseGraph

mg = MaseGraph.from_checkpoint(f"{Path.home()}/tutorial_2_lora")

## Post-Training Quantization (PTQ)

Here, we simply quantize the model and evaluate the effect in its accuracy. First, let's evaluate the model accuracy before quantization (if you're coming from Tutorial 2, this should be the same as the post-LoRA evaluation accuracy). As seen in Tutorial 2, we can use the `get_tokenized_dataset` and `get_trainer` utilities to generate a HuggingFace `Trainer` instance for training and evaluation.

In [ ]:
# from chop.tools import get_tokenized_dataset, get_trainer

# dataset, tokenizer = get_tokenized_dataset(
#     dataset="imdb",
#     checkpoint="bert-base-uncased",
#     return_tokenizer=True,
# )

# trainer = get_trainer(
#     model=mg.model,
#     checkpoint="prajjwal1/bert-tiny",
#     tokenized_dataset=dataset,
#     tokenizer=tokenizer,
#     evaluate_metric="accuracy",
# )

# # Evaluate accuracy
# eval_results = trainer.evaluate()
# print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

To run the quantization pass, we pass a quantization configuration dictionary as argument. This defines the quantization mode, numerical format and precision for each operator in the graph. We'll run the quantization in "by type" mode, meaning nodes are quantized according to their `mase_op`. Other modes include by name and by regex name. We'll quantize all activations, weights and biases in the model to fixed-point with the same precision. This may be sub-optimal, but works as an example. In Tutorial 4, we'll see how to run the `search` flow in `Mase` to find optimal quantization configurations to minimize accuracy loss.

In [ ]:
import chop.passes as passes

mg, _ = passes.prune_transform_pass(
    mg,
    pass_args={
        "weight" : {
            "scope": "local", 
            "granularity": "element", 
            "method": "l1", 
            "sparsity": 0.5,
        }
        "activation" : {
            "scope": "local", # ["local, "global"] are available
            "granularity": "element", # ["element"] are available
            "method": "l1", # ["l1", "random"] are available
            "sparsity": 0.5, # a float between 0.0 and 1.0
    }
    }
)

We can save the current checkpoint for future reference (optional).

In [ ]:
from pathlib import Path

mg.export(f"{Path.home()}/tutorial_3_ptq")

## Quantization-Aware Training (QAT)

You should have seen in the last section that quantization can lead to a significant drop in accuracy. Next, we'll run QAT to evaluate whether this performance gap can be reduced. To run QAT in Mase, all you need to do is include the model back in your training loop after running the quantization pass.

In [ ]:
# Evaluate accuracy
trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

We can see the accuracy of the quantized model can match (or sometimes exceed) the full precision model, with a much lower memory requirement to store the weights. Finally, save the final checkpoint for future tutorials.

In [ ]:
from pathlib import Path

mg.export(f"{Path.home()}/tutorial_3_qat")